In [1]:
!pip install transformers datasets

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/10.5 MB ? eta -:--:--
   ---- ----------------------------------- 1.0/10.5 MB 7.1 MB/s eta 0:00:02
   ------- -------------------------------- 1.8/10.5 MB 4.8 MB/s eta 0:00:02
   --------- ------------------------------ 2.4/10.5 MB 3.6 MB/s eta 0:00:03
   ------------ --------------------------- 3.1/10.5 MB 4.3 MB/s eta 0:00:02
   ----------------- ---------------------- 4.5/10.5 MB 4.2 MB/s eta 0:00:02
   ------------------- -------------------- 5.0/10.5 MB 4.0 MB/s eta 0:00:02
   ---------------------- ----------------- 5.8/10.5 MB 4.0 MB/s eta 0:00:02
   ------------------------- -------------- 6.6/10.5 MB 3.9 MB/s eta 0:00:02
   ---------------------------- ----------- 7.3/10.5 MB 3.9 MB/s eta 0:00:01
   ------------------------------- -------- 8.1/10.5 MB 3.8 MB/s eta 0:00:01
   ---------------------------------- ----- 8.9/10.5 MB 3.8 MB/s eta 0:00:01
   --

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [2]:
!pip install -q tensorflow-text

ERROR: Could not find a version that satisfies the requirement tensorflow-text (from versions: none)
ERROR: No matching distribution found for tensorflow-text


In [1]:
import pyforest as py

In [2]:
df=pd.read_csv("combined_data.csv")

<IPython.core.display.Javascript object>

In [3]:
df.head(10)

,label,text
0,1,ounce feather bowl hummingbird opec moment ala...
1,1,wulvob get your medircations online qnb ikud v...
2,0,computer connection from cnn com wednesday es...
3,1,university degree obtain a prosperous future m...
4,0,thanks for all your answers guys i know i shou...
5,0,larry king live at escapenumber escapenumber p...
6,0,michael pobega wrote i'm not sure if it's the ...
7,0,hi i have this error tr sample escapenumber es...
8,1,works gateway world art explore tattooing full...
9,1,upon this account he is not only very cautious...


In [4]:
df['text'].nunique()

83446

In [5]:
df[['label']].nunique().sum()


2

In [6]:
df.groupby('label').describe()

text                                                               
       count unique                                                top freq
label                                                                      
0      39538  39538   computer connection from cnn com wednesday es...    1
1      43910  43910  ounce feather bowl hummingbird opec moment ala...    1

In [7]:
39538/43910

0.9004327032566614

In [8]:
import tensorflow_hub as hub

In [9]:
from transformers import TFBertModel, BertTokenizer
import tensorflow as tf

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    df["text"],
    df["label"],
    test_size=0.2,
    random_state=42,
    stratify=df["label"]
)


In [ ]:
# Tokenize (convert to list!)
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

train_encodings = tokenizer(X_train.tolist(), truncation=True, padding=True, max_length=128, return_tensors="tf")
test_encodings = tokenizer(X_test.tolist(), truncation=True, padding=True, max_length=128, return_tensors="tf")

# Convert labels
y_train = tf.convert_to_tensor(y_train.values)
y_test = tf.convert_to_tensor(y_test.values)

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    {
        "input_ids": train_encodings["input_ids"],
        "attention_mask": train_encodings["attention_mask"]
    },
    y_train
)).shuffle(1000).batch(16)

test_dataset = tf.data.Dataset.from_tensor_slices((
    {
        "input_ids": test_encodings["input_ids"],
        "attention_mask": test_encodings["attention_mask"]
    },
    y_test
)).batch(16)


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout, Layer
from tensorflow.keras.models import Model
from transformers import TFBertModel, BertTokenizer

# Load BERT model
bert_model = TFBertModel.from_pretrained("bert-base-uncased")

# Custom wrapper layer
class BertLayer(Layer):
    def __init__(self, **kwargs):
        super(BertLayer, self).__init__(**kwargs)
        self.bert = bert_model

    def call(self, inputs):
        input_ids, attention_mask = inputs
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        return outputs.pooler_output

# Inputs
input_ids = Input(shape=(128,), dtype=tf.int32, name="input_ids")
attention_mask = Input(shape=(128,), dtype=tf.int32, name="attention_mask")

# BERT Layer
bert_output = BertLayer()([input_ids, attention_mask])

# Additional layers
x = Dropout(0.3)(bert_output)
x = Dense(256, activation="relu")(x)
x = Dropout(0.3)(x)
x = Dense(64, activation="relu")(x)
output = Dense(1, activation="sigmoid")(x)

# Model
model = Model(inputs=[input_ids, attention_mask], outputs=output)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()

In [ ]:
model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=5
)